# Lab 2 Text Classification
*Prepared by [Guangcan MAI](http://www.comp.hkbu.edu.hk/~csgcmai)*

*For COMP4075/7630, Semester 1, 2016-2017, in Hong Kong Baptist Univeristy*

## Objective:
This lab aims to introduce to you how to classify text acquired from Web. In particular, we will focus on reviews of entertainment such as movie, restaurant and services. With this lab, you should be able to understand the following concepts and able to using Python programming language for their corresponding implementation.

1. Basic pipeline for a pattern recognition system
2. Basic techniques for pre-processing natural language in text
3. Basic representations (features) for natural language in text
4. Basic classifier for text classification
5. Fundmental metris for classification model evaluation

## Basic pipeline for a pattern recognition systems
*Slides from ELEG5040, CUHK, Introduction to Deep Learning, by Prof. Xiaogang Wang 
![alt text](figure/wi_lab2_fig1.png "Basic_Pipleline")

## Basic Naïve Bayes Text Classifier for movie reviews
[Reference](http://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/)

### Bag-of-words model (representations / features)
A simplified version of the bag-of-words model is used here for the movie review analysis. Different from the bag-of-words models that calculate the word frequency in each document, we instead use the model that **whether the words existed** in the document. 
![alt text](figure/wi_lab2_fig2.png "Bag-of-Words")

### [NLTK Package](http://www.nltk.org/)
> NLTK is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, wrappers for industrial-strength NLP libraries, and an active [discussion forum](http://groups.google.com/group/nltk-users).

* Python package that implements many standard NLP data structures, algorithms
* First developed in 2001 as part of a CL course at University of Pennsylvania
* Many contributors since then
- led by Steven Bird, Edward Loper, Ewan Klein
* Open-source

### Let us import the nltk module and download a movie reviews corpus by executing nltk.download(). 

In [1]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> movie_reviews
        /home/comp/csgcmai/nltk_data...
      Package movie_reviews is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Package stopwords is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloa

True

### Import the classification tools, basic utility functions and the corpus, i.e., movie reviews, which are just downloaded.

In [2]:
from nltk.corpus import movie_reviews
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from __future__ import print_function

### Take a look on the movie reviews, the way for reading the corpus might different from corpus to corpus, for details, please refer to the official documents or some useful examples

In [3]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

for w in movie_reviews.words(posids[0]):
    print('%s '%w, end="")

films adapted from comic books have had plenty of success , whether they ' re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there ' s never really been a comic book like from hell before . for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid ' 80s with a 12 - part series called the watchmen . to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . in other words , don ' t dismiss this film because of its source . if you can get past the whole comic book thing , you might find another stumbling block in from hell ' s directors , albert and allen hughes . getting the hughes brothers to direct this seems almos

### Define our simplified bag-of-words model, i.e., whether the words exist. Display word_features

In [4]:
def word_feats(words):
    return dict([(word, True) for word in words])
word_feats(movie_reviews.words(posids[0]))

{u'"': True,
 u"'": True,
 u'(': True,
 u')': True,
 u',': True,
 u'-': True,
 u'.': True,
 u'/': True,
 u'00': True,
 u'102': True,
 u'12': True,
 u'1888': True,
 u'2': True,
 u'30': True,
 u'500': True,
 u'80s': True,
 u':': True,
 u'?': True,
 u'a': True,
 u'abberline': True,
 u'ably': True,
 u'about': True,
 u'absinthe': True,
 u'accent': True,
 u'acting': True,
 u'acts': True,
 u'actually': True,
 u'adapted': True,
 u'after': True,
 u'alan': True,
 u'albert': True,
 u'all': True,
 u'allen': True,
 u'almost': True,
 u'amounts': True,
 u'an': True,
 u'and': True,
 u'another': True,
 u'anyone': True,
 u'anything': True,
 u'apes': True,
 u'appearance': True,
 u'are': True,
 u'arriving': True,
 u'arthouse': True,
 u'as': True,
 u'at': True,
 u'attempt': True,
 u'back': True,
 u'bad': True,
 u'batman': True,
 u'be': True,
 u'because': True,
 u'been': True,
 u'before': True,
 u'befriends': True,
 u'behind': True,
 u'better': True,
 u'big': True,
 u'black': True,
 u'blame': True,
 u'bleak

### Extracting features for negative reviews and positive reviews, respectively

In [5]:
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

### Training / testing partition
* Training samples are used to train the classifier, that is, find the boundary (curve) between the positive and negative reviews
* Testing samples are used to evaluate how good is the found boundary

In [6]:
negcutoff = len(negfeats)*3/4
poscutoff = len(posfeats)*3/4
 
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

### Train the classifier (naïve Bayes) and evaluate the classification accuracy

In [7]:
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))

classifier.show_most_informative_features()

train on 1500 instances, test on 500 instances
accuracy: 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


## Advanced Classifiers and Feature Extraction for Entertainment Reviews
[Reference](http://blog.chapagain.com.np/machine-learning- sentiment-analysis- text- classification-using- python-nltk/)
### Read the data 
* Download the review comments from BU-Moodle and then upload to your [current folder]/data/
* Read the csv data using csv reader and then stored as a list of words

In [8]:
import csv

posdata = []
with open('data/positive-data.csv', 'rb') as myfile:    
    reader = csv.reader(myfile, delimiter=',')
    for val in reader:
        posdata.append(val[0])        
 
negdata = []
with open('data/negative-data.csv', 'rb') as myfile:    
    reader = csv.reader(myfile, delimiter=',')
    for val in reader:
        negdata.append(val[0])

def word_split(data):    
    data_new = []
    for word in data:
        word_filter = [i.lower() for i in word.split()]
        data_new.append(word_filter)
    return data_new

### Preprocessing: filter out the stop words, words that will be excluded from feature words

In [9]:
from nltk.corpus import stopwords
for w in stopwords.words('english'):
    print(w,end=" ") 

i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now d ll m o re ve y ain aren couldn didn doesn hadn hasn haven isn ma mightn mustn needn shan shouldn wasn weren won wouldn 

### Representations: word feature, word feature with filtered words
#### Stopwords that are useful for the sentiment analysis is kept here

In [10]:
#Update the set of stopwords
stopset = set(stopwords.words('english')) - set(('over', 'under', 'below', 'more', 'most', 'no', 'not', 'only', 'such', 'few', 'so', 'too', 'very', 'just', 'any', 'once'))

def word_feats(words):    
    return dict([(word, True) for word in words])

def stopword_filtered_word_feats(words):
    return dict([(word, True) for word in words if word not in stopset])

#### Extend the word feature with bigram
A **bigram** or digram is a sequence of two adjacent elements from a string of tokens, which are typically letters, syllables, or words. A bigram is an n-gram for n=2.

![alt text](figure/wi_lab2_fig3.jpg "N-Gram_model")

In [11]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.metrics import scores
import itertools

def bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    """
    print words
    for ngram in itertools.chain(words, bigrams): 
        if ngram not in stopset: 
            print ngram
    exit()
    """    
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])
    
def bigram_word_feats_stopwords(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    """
    print words
    for ngram in itertools.chain(words, bigrams): 
        if ngram not in stopset: 
            print ngram
    exit()
    """    
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams) if ngram not in stopset])

In [12]:
ws_pos = word_split(posdata)
print("Original word features are:")
print(" ")
for w in ws_pos[1]:
    print(w,end=" ")
print(" ")
print(" ")
print("Extended word features with bigram are:")
print(" ")
ws_pos_with_bg = bigram_word_feats(ws_pos[1])
for w in ws_pos_with_bg.keys():
    print(w ,end=" ")

Original word features are:
 
i dont know why anyone eats ice cream anymore  
 
Extended word features with bigram are:
 
('cream', 'anymore') ('ice', 'cream') dont anymore i ('i', 'dont') ('dont', 'know') eats ice ('know', 'why') anyone ('eats', 'ice') know ('anyone', 'eats') why ('why', 'anyone') cream 

## Train the classifier (naïve Bayes) and evaluate the performance in terms of accuracy, precision, recall, under the One-fold and N-fold cross-validation
![alt text](figure/wi_lab2_fig4.png "TP_NP")
\begin{align}
\text{Precision} & = \frac{True Postives}{True Positives + False Positives} \\
\text{Recall} & = \frac{True Postives}{True Positives + False Negatives} \\
\text{Accuracy} & = \frac{True Postives + True Negatives}{True Positives + True Negatives + False Positives + + False Negatives} \\
\end{align}

### K-fold cross validation
[Wikipedia] In **k-fold cross-validation**, the original sample is randomly partitioned into k equal sized subsamples. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k − 1 subsamples are used as training data. The cross-validation process is then repeated k times (the folds), with each of the k subsamples used exactly once as the validation data. The k results from the folds can then be averaged to produce a single estimation. The advantage of this method over repeated random sub-sampling (see below) is that all observations are used for both training and validation, and each observation is used for validation exactly once. 10-fold cross-validation is commonly used,[6] but in general k remains an unfixed parameter.
![alt text](figure/wi_lab2_fig5.jpg "KFCV")

In [13]:
import collections
import random
# Calculating Precision, Recall & F-measure
def evaluate_classifier(featx):
    
    negfeats = [(featx(f), 'neg') for f in word_split(negdata)]
    posfeats = [(featx(f), 'pos') for f in word_split(posdata)]
        
    negcutoff = len(negfeats)*3/4
    poscutoff = len(posfeats)*3/4
 
    trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
    testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
    
    # using 3 classifiers
    classifier_list = ['nb',]     
        
    for cl in classifier_list:
        if cl == 'nb':
            classifierName = 'Naive Bayes'
            classifier = NaiveBayesClassifier.train(trainfeats)
            
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)
 
        for i, (feats, label) in enumerate(testfeats):
                refsets[label].add(i)
                observed = classifier.classify(feats)
                testsets[observed].add(i)
 
        accuracy = nltk.classify.util.accuracy(classifier, testfeats)
        pos_precision = scores.precision(refsets['pos'], testsets['pos'])
        pos_recall = scores.recall(refsets['pos'], testsets['pos'])
        pos_fmeasure = scores.f_measure(refsets['pos'], testsets['pos'])
        neg_precision = scores.precision(refsets['neg'], testsets['neg'])
        neg_recall = scores.recall(refsets['neg'], testsets['neg'])
        neg_fmeasure =  scores.f_measure(refsets['neg'], testsets['neg'])
        
        print('')
        print( '---------------------------------------')
        print( 'SINGLE FOLD RESULT ' + '(' + classifierName + ')')
        print( '---------------------------------------')
        print( 'accuracy:', accuracy)
        print( 'precision', (pos_precision + neg_precision) / 2)
        print( 'recall', (pos_recall + neg_recall) / 2)
        print( 'f-measure', (pos_fmeasure + neg_fmeasure) / 2)
                
        #classifier.show_most_informative_features()
    
    print( '')
    
    ## CROSS VALIDATION
    
    trainfeats = negfeats + posfeats    
    
    # SHUFFLE TRAIN SET
    # As in cross validation, the test chunk might have only negative or only positive data    
    random.shuffle(trainfeats)    
    n = 5 # 5-fold cross-validation    
    
    for cl in classifier_list:
        
        subset_size = len(trainfeats) / n
        accuracy = []
        pos_precision = []
        pos_recall = []
        neg_precision = []
        neg_recall = []
        pos_fmeasure = []
        neg_fmeasure = []
        cv_count = 1
        for i in range(n):        
            testing_this_round = trainfeats[i*subset_size:][:subset_size]
            training_this_round = trainfeats[:i*subset_size] + trainfeats[(i+1)*subset_size:]
            
            if cl == 'nb':
                classifierName = 'Naive Bayes'
                classifier = NaiveBayesClassifier.train(training_this_round)
                    
            refsets = collections.defaultdict(set)
            testsets = collections.defaultdict(set)
            for i, (feats, label) in enumerate(testing_this_round):
                refsets[label].add(i)
                observed = classifier.classify(feats)
                testsets[observed].add(i)
            
            cv_accuracy = nltk.classify.util.accuracy(classifier, testing_this_round)
            cv_pos_precision = scores.precision(refsets['pos'], testsets['pos'])
            cv_pos_recall = scores.recall(refsets['pos'], testsets['pos'])
            cv_pos_fmeasure = scores.f_measure(refsets['pos'], testsets['pos'])
            cv_neg_precision = scores.precision(refsets['neg'], testsets['neg'])
            cv_neg_recall = scores.recall(refsets['neg'], testsets['neg'])
            cv_neg_fmeasure =  scores.f_measure(refsets['neg'], testsets['neg'])
                    
            accuracy.append(cv_accuracy)
            pos_precision.append(cv_pos_precision)
            pos_recall.append(cv_pos_recall)
            neg_precision.append(cv_neg_precision)
            neg_recall.append(cv_neg_recall)
            pos_fmeasure.append(cv_pos_fmeasure)
            neg_fmeasure.append(cv_neg_fmeasure)
            
            cv_count += 1
                
        print('---------------------------------------')
        print( 'N-FOLD CROSS VALIDATION RESULT ' + '(' + classifierName + ')')
        print( '---------------------------------------')
        print( 'accuracy:', sum(accuracy) / n)
        print( 'precision', (sum(pos_precision)/n + sum(neg_precision)/n) / 2)
        print( 'recall', (sum(pos_recall)/n + sum(neg_recall)/n) / 2)
        print( 'f-measure', (sum(pos_fmeasure)/n + sum(neg_fmeasure)/n) / 2)
        print( '')

In [14]:
evaluate_classifier(bigram_word_feats) 


---------------------------------------
SINGLE FOLD RESULT (Naive Bayes)
---------------------------------------
accuracy: 0.812
precision 0.863372093023
recall 0.812
f-measure 0.805111874077

---------------------------------------
N-FOLD CROSS VALIDATION RESULT (Naive Bayes)
---------------------------------------
accuracy: 0.818
precision 0.858810358329
recall 0.819310322563
f-measure 0.812443284241



In [15]:
evaluate_classifier(bigram_word_feats_stopwords)


---------------------------------------
SINGLE FOLD RESULT (Naive Bayes)
---------------------------------------
accuracy: 0.864
precision 0.893081761006
recall 0.864
f-measure 0.861437141367

---------------------------------------
N-FOLD CROSS VALIDATION RESULT (Naive Bayes)
---------------------------------------
accuracy: 0.858
precision 0.879803670962
recall 0.856867978853
f-measure 0.855152871454

